In [1]:
import os
import numpy as np
import env_ingredient_add as env
import hw
import trajectories as traj
from importlib import reload
from enum import Enum

def get_scene():
    if os.getcwd() == "/datasets/_deepnote_work/manipulation/project": 
        #scene = open("/work/manipulation/project/objects/environment_setup.yaml")
        scene = env.get_environment_set_up(no_scene=True,include_driver=True)
        xmls = [os.getcwd() + "/package.xml", "/work/manipulation/project/package.xml"]
    else:
        #scene = open("objects/environment_setup.yaml") # local setup
        scene = env.get_environment_set_up(no_scene=True,include_driver=True)
        xmls = [os.getcwd() + "/package.xml"]

    return scene

In [ ]:
scenario = hw.LoadScenario(data=get_scene())
meshcat = hw.StartMeshcat()

INFO:drake:Meshcat listening for connections at http://localhost:7001


### trajectory 0: iiwa_painter

In [76]:
reload(env)
reload(traj)
reload(hw)

# at present, trajectory blank doesn't work
diagram,sim = hw.init_diagram(meshcat,scenario)
# hw.print_diagram(diagram)

context = diagram.CreateDefaultContext()
traj0 = hw.create_painter_trajectory(diagram,meshcat,context)

meshcat.Delete()
diagram,sim = hw.init_diagram(meshcat,scenario,traj0)
controller = diagram.GetSubsystemByName("diff_ik_integrator")


In [72]:
# hw.print_diagram(diagram,output_file="planner.png")

In [77]:
hw.fix_base_pos(controller.get_parameters(), 3*[True]) # z is locked to 25 cm.
# hw.create_painter_trajectory(diagram,meshcat)

hw.run_simulation(meshcat,sim,20)

Current state: START
Transitioning to PLAN_IIWA_PAINTER FSM state.
None
Transitioning to EXECUTE_IIWA_PAINTER FSM state.


KeyboardInterrupt: 

### trajectory 1: simple move to bowl

In [73]:
# TODO(mattg): move this into some sort of planner system that can move between states.

reload(traj)

diagram,sim = hw.init_diagram(meshcat,scenario)
traj1 = traj.traj_linear_move_to_bowl_0(diagram)
meshcat.Delete()

diagram,sim = hw.init_diagram(meshcat,scenario,traj1)

# problem: we're missing a way to actually pass new trajectories into the robot.

# pose_source = diagram.GetSubsystemByName("hw.PoseTrajectorySource@00007fc3283167c0")

In [55]:
hw.run_simulation(meshcat,sim,20)

## Plumbing in a State Machine (WIP)

## scratch code

### gets the name of the robot's joints

In [ ]:

# plant = diagram.GetSubsystemByName("station").GetSubsystemByName("plant")

# for index in plant.GetJointIndices():
#     print(plant.get_joint(index).name())

iiwa_base_x
iiwa_base_y
iiwa_base_z
iiwa_joint_1
iiwa_joint_2
iiwa_joint_3
iiwa_joint_4
iiwa_joint_5
iiwa_joint_6
iiwa_joint_7
iiwa_joint_ee
tool0_joint
left_finger_sliding_joint
right_finger_sliding_joint
iiwa_link_7_welds_to_body


In [ ]:


print(controller.get_parameters().get_joint_position_limits())

print(f"after running: {controller.get_parameters().get_joint_position_limits()}")

hw.run_simulation(meshcat,sim,10)